In this I am training 2 models
Reference:
https://heartbeat.fritz.ai/using-aspect-based-sentiment-analysis-to-understand-user-generated-content-2cfd5d3e25bb

In [4]:
# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
nltk.download('stopwords')

#Spacy
import spacy
nlp = spacy.load('en')

# Other
import re
import json
import string
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

#Keras
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Activation

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jayanthbhargav/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


In [5]:
# Aspects common in test and training data
common_aspects = {'BATTERY#OPERATION_PERFORMANCE',
 'BATTERY#QUALITY',
 'COMPANY#GENERAL',
 'CPU#DESIGN_FEATURES',
 'CPU#MISCELLANEOUS',
 'CPU#OPERATION_PERFORMANCE',
 'DISPLAY#DESIGN_FEATURES',
 'DISPLAY#GENERAL',
 'DISPLAY#OPERATION_PERFORMANCE',
 'DISPLAY#QUALITY',
 'DISPLAY#USABILITY',
 'FANS_COOLING#OPERATION_PERFORMANCE',
 'GRAPHICS#DESIGN_FEATURES',
 'GRAPHICS#MISCELLANEOUS',
 'HARDWARE#QUALITY',
 'HARD_DISC#DESIGN_FEATURES',
 'HARD_DISC#QUALITY',
 'KEYBOARD#DESIGN_FEATURES',
 'KEYBOARD#GENERAL',
 'KEYBOARD#OPERATION_PERFORMANCE',
 'KEYBOARD#QUALITY',
 'KEYBOARD#USABILITY',
 'LAPTOP#CONNECTIVITY',
 'LAPTOP#DESIGN_FEATURES',
 'LAPTOP#MISCELLANEOUS',
 'LAPTOP#OPERATION_PERFORMANCE',
 'LAPTOP#PORTABILITY',
 'LAPTOP#PRICE',
 'LAPTOP#QUALITY',
 'LAPTOP#USABILITY',
 'MEMORY#DESIGN_FEATURES',
 'MOTHERBOARD#QUALITY',
 'MOUSE#DESIGN_FEATURES',
 'MOUSE#GENERAL',
 'MOUSE#OPERATION_PERFORMANCE',
 'MOUSE#QUALITY',
 'MOUSE#USABILITY',
 'MULTIMEDIA_DEVICES#DESIGN_FEATURES',
 'MULTIMEDIA_DEVICES#GENERAL',
 'MULTIMEDIA_DEVICES#QUALITY',
 'MULTIMEDIA_DEVICES#USABILITY',
 'OS#DESIGN_FEATURES',
 'OS#GENERAL',
 'OS#MISCELLANEOUS',
 'OS#OPERATION_PERFORMANCE',
 'OS#QUALITY',
 'OS#USABILITY',
 'PORTS#OPERATION_PERFORMANCE',
 'POWER_SUPPLY#DESIGN_FEATURES',
 'SHIPPING#QUALITY',
 'SOFTWARE#DESIGN_FEATURES',
 'SOFTWARE#GENERAL',
 'SOFTWARE#OPERATION_PERFORMANCE',
 'SOFTWARE#PRICE',
 'SOFTWARE#QUALITY',
 'SOFTWARE#USABILITY',
 'SUPPORT#PRICE',
 'SUPPORT#QUALITY',
 'WARRANTY#GENERAL'}

In [6]:
#load data
import pandas as pd
reviews_train = pd.read_csv("output_laptop.csv", encoding = "ISO-8859-1").astype(str)
reviews_train = reviews_train.rename(columns = {'sentences__sentence__text':'reviews', 'sentences__sentence__Opinions__Opinion__category': 'aspect_category', 'sentences__sentence__Opinions__Opinion__polarity': 'sentiment'  })
reviews_train = reviews_train[reviews_train['aspect_category'] != 'nan']

# show first 5 records
reviews_train.head()

,reviews,aspect_category,sentiment
1,This computer is absolutely AMAZING!!!,LAPTOP#GENERAL,positive
2,10 plus hours of battery...,BATTERY#OPERATION_PERFORMANCE,positive
3,super fast processor and really nice graphics ...,CPU#OPERATION_PERFORMANCE,positive
4,super fast processor and really nice graphics ...,GRAPHICS#GENERAL,positive
5,and plenty of storage with 250 gb(though I wil...,HARD_DISC#DESIGN_FEATURES,positive


In [7]:
# Preprocess Training Data
reviews_train = reviews_train.reset_index(drop = True)

for i in range(len(reviews_train)):
    aspect = reviews_train['aspect_category'][i]
    if aspect not in common_aspects:
        aspect = 'nan'
        reviews_train['aspect_category'][i] = aspect

reviews_train = reviews_train[reviews_train['aspect_category'] != 'nan']
display(reviews_train)

,reviews,aspect_category,sentiment
1,10 plus hours of battery...,BATTERY#OPERATION_PERFORMANCE,positive
2,super fast processor and really nice graphics ...,CPU#OPERATION_PERFORMANCE,positive
4,and plenty of storage with 250 gb(though I wil...,HARD_DISC#DESIGN_FEATURES,positive
5,This computer is really fast and I'm shocked a...,LAPTOP#OPERATION_PERFORMANCE,positive
6,This computer is really fast and I'm shocked a...,LAPTOP#USABILITY,positive
...,...,...,...
2903,"Price is great, wish it didn't have Windows 8,...",OS#GENERAL,negative
2904,"I would recommend it, for anybody needing a re...",LAPTOP#QUALITY,positive
2905,I bought this for my daughter to use for schoo...,LAPTOP#OPERATION_PERFORMANCE,neutral
2906,I bought this for my daughter to use for schoo...,LAPTOP#DESIGN_FEATURES,positive


In [8]:
# reviews_train.columns
print(reviews_train.groupby('aspect_category').size().sort_values(ascending=False))

#how many categories
print("number of categories",reviews_train.aspect_category.nunique())

aspect_category
LAPTOP#OPERATION_PERFORMANCE          278
LAPTOP#DESIGN_FEATURES                253
LAPTOP#QUALITY                        224
LAPTOP#MISCELLANEOUS                  142
LAPTOP#USABILITY                      141
SUPPORT#QUALITY                       138
LAPTOP#PRICE                          136
COMPANY#GENERAL                        90
BATTERY#OPERATION_PERFORMANCE          86
LAPTOP#CONNECTIVITY                    55
DISPLAY#QUALITY                        53
LAPTOP#PORTABILITY                     51
OS#GENERAL                             35
SOFTWARE#GENERAL                       31
KEYBOARD#DESIGN_FEATURES               29
MOUSE#OPERATION_PERFORMANCE            28
DISPLAY#DESIGN_FEATURES                28
MULTIMEDIA_DEVICES#QUALITY             26
DISPLAY#GENERAL                        24
MOUSE#USABILITY                        23
KEYBOARD#USABILITY                     21
OS#USABILITY                           21
KEYBOARD#QUALITY                       18
CPU#OPERATION_PERF

In [16]:
display(reviews_train)

,reviews,aspect_category,sentiment
1,10 plus hours of battery...,BATTERY#OPERATION_PERFORMANCE,positive
2,super fast processor and really nice graphics ...,CPU#OPERATION_PERFORMANCE,positive
4,and plenty of storage with 250 gb(though I wil...,HARD_DISC#DESIGN_FEATURES,positive
5,This computer is really fast and I'm shocked a...,LAPTOP#OPERATION_PERFORMANCE,positive
6,This computer is really fast and I'm shocked a...,LAPTOP#USABILITY,positive
...,...,...,...
2903,"Price is great, wish it didn't have Windows 8,...",OS#GENERAL,negative
2904,"I would recommend it, for anybody needing a re...",LAPTOP#QUALITY,positive
2905,I bought this for my daughter to use for schoo...,LAPTOP#OPERATION_PERFORMANCE,neutral
2906,I bought this for my daughter to use for schoo...,LAPTOP#DESIGN_FEATURES,positive


In [10]:
# Test Data
reviews_test = pd.read_csv('laptop_test.csv')
reviews_test = reviews_test.drop(columns = ['rid', 'id', 'OutOfScope'])
reviews_test = reviews_test.rename(columns = {'text':'reviews', 'category': 'aspect_category', 'polarity': 'sentiment'  })
reviews_test = reviews_test[reviews_test['aspect_category'].notnull()]

In [11]:
reviews_test = reviews_test.reset_index(drop = True)

for i in range(len(reviews_test)):
    aspect = reviews_test['aspect_category'][i]
    if aspect not in common_aspects:
        aspect = 'nan'
        reviews_test['aspect_category'][i] = aspect

reviews_test = reviews_test[reviews_test['aspect_category'] != 'nan']
display(reviews_test)

,reviews,aspect_category,sentiment
1,"Works well, fast and no reboots.",LAPTOP#OPERATION_PERFORMANCE,positive
2,Glad I did so far.,COMPANY#GENERAL,positive
4,s.... L .... o..... w.... rea......llllyy slow.,LAPTOP#OPERATION_PERFORMANCE,negative
5,like seriously - really slow.,LAPTOP#OPERATION_PERFORMANCE,negative
6,impossible to use.,LAPTOP#USABILITY,negative
...,...,...,...
795,To go left I have to go right and vice versa a...,MOUSE#OPERATION_PERFORMANCE,negative
796,To go left I have to go right and vice versa a...,MOUSE#USABILITY,negative
797,When I attach a mouse it's fine but the touchp...,MOUSE#USABILITY,negative
798,I went through the settings and there isn't a ...,MOUSE#USABILITY,negative


In [15]:
# Model architecture for Aspects
absa_model = Sequential()
absa_model.add(Dense(512, input_shape=(6000,), activation='relu'))
absa_model.add((Dense(256, activation='relu')))
absa_model.add((Dense(128, activation='relu')))
absa_model.add(Dense(59, activation='softmax'))

#compile model
absa_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [18]:
# convert text to TfIDF vectors
from keras.preprocessing.text import Tokenizer

# training examples
vocab_size = 6000 # We set a maximum size for the vocabulary
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(reviews_train.reviews)
reviews_tokenized = pd.DataFrame(tokenizer.sequences_to_matrix(tokenizer.texts_to_sequences(reviews_train.reviews), mode ='tfidf'))

#testing examples
reviews_test_tokenized = pd.DataFrame(tokenizer.texts_to_matrix(reviews_train.reviews))
reviews_test_tokenized = pd.DataFrame(tokenizer.sequences_to_matrix(tokenizer.texts_to_sequences(reviews_test.reviews), mode ='tfidf'))

In [29]:
# One Hot Encoding Sentiment labels
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

label_encoder = LabelEncoder()
integer_category = label_encoder.fit_transform(reviews_train.sentiment)
# Training Examples
encoded_y = to_categorical(integer_category)

# Testing Example
integer_category = label_encoder.transform(reviews_test.sentiment)
encoded_y_test = to_categorical(integer_category)

In [26]:
# model architecture for sentiments
sentiment_model = Sequential()
sentiment_model.add(Dense(512, input_shape=(6000,), activation='relu'))
sentiment_model.add((Dense(256, activation='relu')))
sentiment_model.add((Dense(128, activation='relu')))
sentiment_model.add(Dense(3, activation='softmax'))
#compile model
sentiment_model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [27]:
# One Hot Encoding for aspects
label_encoder = LabelEncoder()
integer_aspects = label_encoder.fit_transform(reviews_train.aspect_category)
encoded_aspect = to_categorical(integer_aspects)

integer_aspects = label_encoder.transform(reviews_test.aspect_category)
encoded_aspect_test = to_categorical(integer_aspects)

In [28]:
#fit aspect classifier
absa_model.fit(reviews_tokenized, encoded_aspect, epochs=100, verbose=1)
#fit sentiment classifier
sentiment_model.fit(reviews_tokenized, encoded_y, epochs=100, verbose=1)

Epoch 1/100
2189/2189 [==============================] - 3s 1ms/step - loss: 0.4419 - accuracy: 0.6843
Epoch 2/100
2189/2189 [==============================] - 3s 1ms/step - loss: 0.4472 - accuracy: 0.6871
Epoch 3/100
2189/2189 [==============================] - 3s 1ms/step - loss: 0.4384 - accuracy: 0.6843
Epoch 4/100
2189/2189 [==============================] - 3s 1ms/step - loss: 0.4470 - accuracy: 0.6761
Epoch 5/100
2189/2189 [==============================] - 3s 1ms/step - loss: 0.4458 - accuracy: 0.6802
Epoch 6/100
2189/2189 [==============================] - 3s 1ms/step - loss: 0.4510 - accuracy: 0.6852
Epoch 7/100
2189/2189 [==============================] - 3s 1ms/step - loss: 0.4464 - accuracy: 0.6784
Epoch 8/100
2189/2189 [==============================] - 3s 1ms/step - loss: 0.4425 - accuracy: 0.6871
Epoch 9/100
2189/2189 [==============================] - 3s 1ms/step - loss: 0.4405 - accuracy: 0.6807
Epoch 10/100
2189/2189 [==============================] - 3s 1ms/step - l

2189/2189 [==============================] - 3s 1ms/step - loss: 0.4235 - accuracy: 0.6971
Epoch 80/100
2189/2189 [==============================] - 3s 1ms/step - loss: 0.4213 - accuracy: 0.6907
Epoch 81/100
2189/2189 [==============================] - 3s 1ms/step - loss: 0.4203 - accuracy: 0.6839
Epoch 82/100
2189/2189 [==============================] - 3s 1ms/step - loss: 0.4178 - accuracy: 0.6871
Epoch 83/100
2189/2189 [==============================] - 3s 1ms/step - loss: 0.4219 - accuracy: 0.6871
Epoch 84/100
2189/2189 [==============================] - 3s 1ms/step - loss: 0.4230 - accuracy: 0.6830
Epoch 85/100
2189/2189 [==============================] - 3s 1ms/step - loss: 0.4195 - accuracy: 0.6798
Epoch 86/100
2189/2189 [==============================] - 3s 1ms/step - loss: 0.4219 - accuracy: 0.6852
Epoch 87/100
2189/2189 [==============================] - 3s 1ms/step - loss: 0.4229 - accuracy: 0.6939
Epoch 88/100
2189/2189 [==============================] - 3s 1ms/step - loss:

2189/2189 [==============================] - 1s 573us/step - loss: 0.0868 - accuracy: 0.9520
Epoch 57/100
2189/2189 [==============================] - 1s 577us/step - loss: 0.0867 - accuracy: 0.9456
Epoch 58/100
2189/2189 [==============================] - 1s 578us/step - loss: 0.0865 - accuracy: 0.9484
Epoch 59/100
2189/2189 [==============================] - 1s 580us/step - loss: 0.0876 - accuracy: 0.9479
Epoch 60/100
2189/2189 [==============================] - 1s 577us/step - loss: 0.0860 - accuracy: 0.9493
Epoch 61/100
2189/2189 [==============================] - 1s 583us/step - loss: 0.0884 - accuracy: 0.9438
Epoch 62/100
2189/2189 [==============================] - 1s 586us/step - loss: 0.0850 - accuracy: 0.9456
Epoch 63/100
2189/2189 [==============================] - 1s 580us/step - loss: 0.0858 - accuracy: 0.9502
Epoch 64/100
2189/2189 [==============================] - 1s 593us/step - loss: 0.0858 - accuracy: 0.9488
Epoch 65/100
2189/2189 [==============================] - 1

In [24]:
aspect_score = absa_model.evaluate(reviews_test_tokenized, encoded_aspect_test)
sentiment_score = sentiment_model.evaluate(reviews_test_tokenized, encoded_y_test)
print('Aspect Prediction Accuracy: ', aspect_score[1])
print('Sentiment Prediction Accuracy: ', sentiment_score[1])

630/630 [==============================] - 0s 165us/step
Aspect Prediction Accuracy:  0.35873016715049744
Sentiment Prediction Accuracy:  0.7158730030059814
